In [ ]:
pip install plotly

In [ ]:
pip install plotly.express

In [ ]:
pip install nbformat

In [243]:
import mysql.connector
import pandas as pd 
import plotly.express as px
import nbformat
import requests
import json
import streamlit as st

In [244]:
#DATAFRAME CREATION
mydb = mysql.connector.connect(
                host="localhost",
                user="root",
                password="",
                database='phonepe',
    )
mycursor = mydb.cursor(buffered=True)


#aggre_transaction_df
mycursor.execute('SELECT * FROM aggregated_transaction')
mydb.commit()
table1=mycursor.fetchall()

Aggre_transaction=pd.DataFrame(table1, columns=('States','Years','Quarter','Transaction_type','Transaction_count','Transaction_amount'))

#aggre_user_df
mycursor.execute('SELECT * FROM aggregated_user')
mydb.commit()
table2=mycursor.fetchall()

Aggre_user=pd.DataFrame(table2,columns=('States','Years','Quarter','Brands','Transaction_count','Percentage'))

#map_transaction_df
mycursor.execute('SELECT * FROM map_transaction')
mydb.commit()
table3=mycursor.fetchall()

map_transaction=pd.DataFrame(table3,columns=('States','Years','Quarter','Districts','Transaction_count','Transaction_amount'))

#map_user_df
mycursor.execute('SELECT * FROM map_user')
mydb.commit()
table4=mycursor.fetchall()

map_user=pd.DataFrame(table4,columns=('States','Years','Quarter','Districts','RegisteredUsers','AppOpens'))

#top_transaction_df
mycursor.execute('SELECT * FROM top_transaction')
mydb.commit()
table5=mycursor.fetchall()
map_user=pd.DataFrame(table5,columns=('States','Years','Quarter','Pincodes','Transaction_count','Transaction_amount'))

#map_user_df
mycursor.execute('SELECT * FROM top_user')
mydb.commit()
table6=mycursor.fetchall()
top_user=pd.DataFrame(table6,columns=('States','Years','Quarter','Pincodes','RegisteredUsers'))

In [254]:
def Transaction_amount_count_Y(df, year):

    tacy =df[df['Years'] == year]
    tacy.reset_index(drop=True,inplace=True)

    tacyg=tacy.groupby('States')[['Transaction_count','Transaction_amount']].sum()
    tacyg.reset_index(inplace=True)

    fig_amount=px.bar(tacyg, x='States', y='Transaction_amount',title=f'{year} TRANSACTION AMOUNT',
                    color_discrete_sequence=px.colors.sequential.Bluered)
    fig_amount.show()

    fig_count=px.bar(tacyg, x='States', y='Transaction_count',title=f'{year} TRANSACTION COUNT',
                    color_discrete_sequence=px.colors.sequential.Bluered_r)
    fig_count.show()
    
    url= "https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson"
    response= requests.get(url)
    data1=json.loads(response.content)
    states_name=[]
    for feature in data1['features']:
        states_name.append(feature['properties']['ST_NM'])
        
    states_name.sort()

    fig_india_1=px.choropleth(tacyg, geojson=data1, locations= 'States', featureidkey= 'properties.ST_NM',
                              color='Transaction_amount', color_continuous_scale= 'rainbow',
                              range_color=(tacyg['Transaction_amount'].min(),tacyg['Transaction_amount'].max()),
                              hover_name='States', title=f'{year} TRANSACTION AMOUNT', fitbounds= 'locations',
                              height= 600, width= 600)
    fig_india_1.update_geos(visible=False)
    fig_india_1.show()

    fig_india_2=px.choropleth(tacyg, geojson=data1, locations= 'States', featureidkey= 'properties.ST_NM',
                              color='Transaction_count', color_continuous_scale= 'rainbow',
                              range_color=(tacyg['Transaction_count'].min(),tacyg['Transaction_count'].max()),
                              hover_name='States', title=f'{year} TRANSACTION COUNT', fitbounds= 'locations',
                              height= 600, width= 600)
    fig_india_2.update_geos(visible=False)
    fig_india_2.show()
    
    


In [255]:
tacy

NameError: name 'tacy' is not defined

In [246]:
def Transaction_amount_count_Y_Q(df, quarter):
    
    tacy =df[df['Quarter'] == quarter]
    tacy.reset_index(drop=True,inplace=True)

    tacyg=tacy.groupby('States')[['Transaction_count','Transaction_amount']].sum()
    tacyg.reset_index(inplace=True)
    
    # col1,col2=st.columns(2)
    # with col1:
        

    fig_amount=px.bar(tacyg, x='States', y='Transaction_amount',title=f'{tacy['Years'].min()} YEAR {quarter} QUARTER TRANSACTION AMOUNT',
                        color_discrete_sequence=px.colors.sequential.Bluered)
    fig_amount.show()
    # with col2:
            
        # col1,col2=st.columns(2)
    # with col1:
                
    url= "https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson"
    response= requests.get(url)
    data1=json.loads(response.content)
    states_name=[]
    for feature in data1['features']:
        states_name.append(feature['properties']['ST_NM'])
            
    states_name.sort()

    fig_india_1=px.choropleth(tacyg, geojson=data1, locations= 'States', featureidkey= 'properties.ST_NM',
                                color='Transaction_amount', color_continuous_scale= 'rainbow',
                                range_color=(tacyg['Transaction_amount'].min(),tacyg['Transaction_amount'].max()),
                                hover_name='States', title=f'{tacy['Years'].min()} YEAR {quarter} QUARTER TRANSACTION AMOUNT', fitbounds= 'locations',
                                height= 600, width= 600)
    fig_india_1.update_geos(visible=False)
    fig_india_1.show()

    # with col2:
    fig_india_2=px.choropleth(tacyg, geojson=data1, locations= 'States', featureidkey= 'properties.ST_NM',
                                color='Transaction_count', color_continuous_scale= 'rainbow',
                                range_color=(tacyg['Transaction_count'].min(),tacyg['Transaction_count'].max()),
                                hover_name='States', title=f'{tacy['Years'].min()} YEAR {quarter} QUARTER TRANSACTION COUNT', fitbounds= 'locations',
                                height= 600, width= 600)
    fig_india_2.update_geos(visible=False)
    fig_india_2.show()



In [247]:
def Aggre_Tran_Transaction_type(df, state):

    tacy =df[df['States'] == state]
    tacy.reset_index(drop=True,inplace=True)

    tacyg=tacy.groupby('Transaction_type')[['Transaction_count','Transaction_amount']].sum()
    tacyg.reset_index(inplace=True)


    fig_pie_1=px.pie(data_frame=tacyg, names='Transaction_type', values='Transaction_amount',
                    width=600,title=f'{state.upper()} TRANSACTION AMOUNT',hole=0.5)
    fig_pie_1.show()

    fig_pie_2=px.pie(data_frame=tacyg, names='Transaction_type', values='Transaction_count',
                    width=600,title=f'{state.upper()} TRANSACTION COUNT',hole=0.5)
    fig_pie_2.show()

In [248]:
#AGGRE_USER_ANALYSIS_1
def Aggre_user_plot_1(df, year):
    aguy= df[df['Years']== year]
    aguy.reset_index(drop= True, inplace= True)

    aguyg=pd.DataFrame(aguy.groupby('Brands')['Transaction_count'].sum())
    aguyg.reset_index(inplace= True)

    fig_bar_1=px.bar(aguyg, x='Brands', y='Transaction_count', title= f'{year} BRANDS AND TRANSACTION COUNT',
                    width=900, color_discrete_sequence=px.colors.sequential.Bluered_r, hover_name='Brands')
    fig_bar_1.show()
    
    return aguy

In [249]:
#AGGRE_USER_ANALYSIS_2
def Aggre_user_plot_2(df, quarter):
    aguyq= df[df['Quarter']== quarter]
    aguyq.reset_index(drop= True, inplace= True)
    
    aguyqg=pd.DataFrame(aguyq.groupby('Brands')['Transaction_count'].sum())
    aguyqg.reset_index(inplace= True)


    fig_bar_1=px.bar(aguyg, x='Brands', y='Transaction_count', title= f'{quarter} QUARTER, BRANDS AND TRANSACTION COUNT',
                        width=900, color_discrete_sequence=px.colors.sequential.Bluered_r, hover_name='Brands')
    fig_bar_1.show()
    return aguyq

In [250]:
#AGGRE_USER_ANALYSIS_3
def Aggre_user_plot_3(df, state):
    auyqs=df[df['States'] == state]
    auyqs.reset_index(drop= True, inplace= True)
    
    fig_line_1=px.line(auyqs, x= 'Brands', y='Transaction_count', hover_name= 'Percentage',
                    title=f'{state.upper()} BRANDS, TRANSACTION COUNT, PERCENTAGE', width= 1000, markers=True)
    fig_line_1.show()

### Aggre_Transaction

In [ ]:
Aggre_tran_tac_Y=Transaction_amount_count_Y(Aggre_transaction, 2021)

In [ ]:
Aggre_tran_tac_Y_Q=Transaction_amount_count_Y_Q(Aggre_tran_tac_Y, 3)

In [182]:
Aggre_tran_tac_Y=Transaction_amount_count_Y(Aggre_transaction, 2021)

In [ ]:
Aggre_Tran_Transaction_type(Aggre_tran_tac_Y, 'Tamil Nadu')

In [ ]:
Aggre_tran_tac_Y_Q=Transaction_amount_count_Y_Q(Aggre_tran_tac_Y, 3)

In [ ]:
Aggre_tran_tac_Y["States"].unique()

array(['Andaman & Nicobar', 'Andhra Pradesh', 'Arunachal Pradesh',
       'Assam', 'Bihar', 'Chandigarh', 'Chhattisgarh',
       'Dadra and Nagar Haveli and Daman and Diu', 'Delhi', 'Goa',
       'Gujarat', 'Haryana', 'Himachal Pradesh', 'Jammu & Kashmir',
       'Jharkhand', 'Karnataka', 'Kerala', 'Ladakh', 'Lakshadweep',
       'Madhya Pradesh', 'Maharashtra', 'Manipur', 'Meghalaya', 'Mizoram',
       'Nagaland', 'Odisha', 'Puducherry', 'Punjab', 'Rajasthan',
       'Sikkim', 'Tamil Nadu', 'Telangana', 'Tripura', 'Uttar Pradesh',
       'Uttarakhand', 'West Bengal'], dtype=object)

In [209]:
##AGGREGATED USER

In [ ]:
Aggre_user_Y= Aggre_user_plot_1(Aggre_user, 2020)

In [ ]:
Aggre_user_Y_Q=Aggre_user_plot_2(Aggre_user_Y, 3)

In [ ]:
Aggre_user_plot_3(Aggre_user_Y_Q,'Tamil Nadu')

In [ ]:
Transaction_amount_count_Y_Q(tacy, 3)

###map_transaction

In [ ]:
map_tran_tac_Y= Transaction_amount_count_Y(map_transaction, 2018)

In [253]:
tacy

NameError: name 'tacy' is not defined

In [238]:
Aggre_Tran_Transaction_type(Aggre_tran_tac_Y, 'Bihar')
    

TypeError: 'NoneType' object is not subscriptable

In [236]:
map_tran_tac_Y

In [173]:
def map_tran_Districts(df, state):
    
    tacy= df[df['States'] == state]
    tacy.reset_index(drop= True, inplace= True)

    tacyg=tacy.groupby('Districts')[['Transaction_count','Transaction_amount']].sum()
    tacyg.reset_index(inplace=True)


    fig_pie_1=px.pie(data_frame=tacyg, names='Districts', values='Transaction_amount',
                    width=600,title=f'{state.upper()} TRANSACTION AMOUNT',hole=0.5)
    fig_pie_1.show()

    fig_pie_2=px.pie(data_frame=tacyg, names='Districts', values='Transaction_count',
                    width=600,title=f'{state.upper()} TRANSACTION COUNT',hole=0.5)
    fig_pie_2.show()


In [ ]:
tac_Y=Transaction_amount_count_Y(Aggre_transaction, 2023)

In [ ]:
map_tran_Districts(map_tran_tac_Y, 'West Bengal')

In [ ]:
Aggre_tran_tac_Y= Transaction_amount_count_Y(Aggre_transaction, 2018)

In [ ]:
def Aggre_Tran_Transaction_type(df, state):
    
    tacy =df[df['States'] == state]
    tacy.reset_index(drop=True,inplace=True)

    tacyg=tacy.groupby('Transaction_type')[['Transaction_count','Transaction_amount']].sum()
    tacyg.reset_index(inplace=True)


    fig_pie_1=px.pie(data_frame=tacyg, names='Transaction_type', values='Transaction_amount',
                    width=600,title=f'{state.upper()} TRANSACTION AMOUNT',hole=0.5)
    fig_pie_1.show()

    fig_pie_2=px.pie(data_frame=tacyg, names='Transaction_type', values='Transaction_count',
                    width=600,title=f'{state.upper()} TRANSACTION COUNT',hole=0.5)
    fig_pie_2.show()

In [231]:
map_transaction

,States,Years,Quarter,Districts,Transaction_count,Transaction_amount
0,Andaman & Nicobar,2018,1,north and middle andaman district,442,9.316630e+05
1,Andaman & Nicobar,2018,1,south andaman district,5688,1.256020e+07
2,Andaman & Nicobar,2018,1,nicobars district,528,1.139850e+06
3,Andaman & Nicobar,2018,2,north and middle andaman district,825,1.317860e+06
4,Andaman & Nicobar,2018,2,south andaman district,9395,2.394820e+07
...,...,...,...,...,...,...
17559,West Bengal,2023,4,nadia district,36242432,6.879570e+10
17560,West Bengal,2023,4,birbhum district,20708496,3.690500e+10
17561,West Bengal,2023,4,purba medinipur district,40572042,7.981990e+10
17562,West Bengal,2023,4,maldah district,34654950,6.416730e+10


In [256]:
def map_tran_plot_1(df,state):
    miys= df[df["States"] == state]
    miysg= miys.groupby("Districts")[["Transaction_count","Transaction_amount"]].sum()
    miysg.reset_index(inplace= True)

    col1,col2= st.columns(2)
    with col1:
        fig_map_bar_1= px.bar(miysg, x= "Districts", y= "Transaction_amount",
                              width=600, height=500, title= f"{state.upper()} DISTRICTS TRANSACTION AMOUNT",
                              color_discrete_sequence= px.colors.sequential.Mint_r)
        st.plotly_chart(fig_map_bar_1)

    with col2:
        fig_map_bar_1= px.bar(miysg, x= "Districts", y= "Transaction_count",
                              width=600, height= 500, title= f"{state.upper()} DISTRICTS TRANSACTION COUNT",
                              color_discrete_sequence= px.colors.sequential.Mint)
        
        st.plotly_chart(fig_map_bar_1)

def map_tran_plot_2(df,state):
    miys= df[df["States"] == state]
    miysg= miys.groupby("Districts")[["Transaction_count","Transaction_amount"]].sum()
    miysg.reset_index(inplace= True)

    col1,col2= st.columns(2)
    with col1:
        fig_map_pie_1= px.pie(miysg, names= "Districts", values= "Transaction_amount",
                              width=600, height=500, title= f"{state.upper()} DISTRICTS TRANSACTION AMOUNT",
                              hole=0.5,color_discrete_sequence= px.colors.sequential.Mint_r)
        st.plotly_chart(fig_map_pie_1)

    with col2:
        fig_map_pie_1= px.pie(miysg, names= "Districts", values= "Transaction_count",
                              width=600, height= 500, title= f"{state.upper()} DISTRICTS TRANSACTION COUNT",
                              hole=0.5,  color_discrete_sequence= px.colors.sequential.Oranges_r)
        
        st.plotly_chart(fig_map_pie_1)
